In [1]:
import urllib2
import pandas as pd
import requests
from bokeh.embed import components 
from bokeh.plotting import figure
import json
from urllib2 import Request, urlopen
from pandas.io.json import json_normalize
from time import sleep
import numpy as np

'http://download.open.fda.gov/device/510k/device-510k-0001-of-0001.json.zip'

In [2]:


!pwd

/Users/taftmphee/Desktop


In [3]:
outputjson = json.load(response)
type(outputjson)

NameError: name 'response' is not defined

In [4]:
#raw_data2.json()
#returns this metadata:  u'results': {u'limit': 25, u'skip': 0, u'total': 5899773}}
#getting the first 5000 records to explore 
urls = []
iter = range(0, 5100, 100)
for i, item in enumerate(iter):
    if i == 0:
        skip = iter[0]
    else: 
        skip = iter[i-1]
    url = 'https://api.fda.gov/device/event.json?api_key=a1lInp8iF29I0xq4d4FVawMdkArMywT3jYKOZeRx&limit=100&skip=%s' % (skip)
    urls.append(url)

In [5]:
#manually dropping the first one
del(urls[0])

In [65]:
#urls

In [6]:
responses = []

In [7]:
session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
for url in urls:
    #print url
    sleep(.25)
    api_url = url
    responses.append(session.get(api_url))


In [107]:
pony['results'][0]['device'][0].keys()

[u'lot_number',
 u'manufacturer_d_country',
 u'brand_name',
 u'manufacturer_d_postal_code',
 u'date_received',
 u'baseline_510_k__exempt_flag',
 u'device_operator',
 u'generic_name',
 u'openfda',
 u'model_number',
 u'device_event_key',
 u'device_evaluated_by_manufacturer',
 u'manufacturer_d_zip_code',
 u'baseline_510_k__flag',
 u'expiration_date_of_device',
 u'device_sequence_number',
 u'device_age_text',
 u'manufacturer_d_zip_code_ext',
 u'baseline_510_k__number',
 u'manufacturer_d_state',
 u'catalog_number',
 u'implant_flag',
 u'manufacturer_d_address_2',
 u'other_id_number',
 u'manufacturer_d_address_1',
 u'manufacturer_d_city',
 u'manufacturer_d_name',
 u'device_availability',
 u'date_removed_flag',
 u'device_report_product_code']

In [161]:
#dicts[10][0]['adverse_event_flag'] == "N"
pony['results'][0]['event_location']

u''

In [99]:
pony = responses[0].json()
#returns this metadata: u'results': {u'limit': 100, u'skip': 0, u'total': 4619711}
pony['results'][0].keys()
#that is one adverse event--from it we want: 
'number_patients_in_event'
'adverse_event_flag'
'event_type'?
'event_location'?
##plot geographic and normalize for population by state?

[u'manufacturer_contact_t_name',
 u'manufacturer_g1_name',
 u'number_patients_in_event',
 u'manufacturer_g1_city',
 u'event_type',
 u'manufacturer_city',
 u'report_number',
 u'product_problem_flag',
 u'manufacturer_country',
 u'manufacturer_g1_address_2',
 u'manufacturer_g1_address_1',
 u'device',
 u'device_date_of_manufacturer',
 u'manufacturer_contact_city',
 u'distributor_address_2',
 u'patient',
 u'single_use_flag',
 u'manufacturer_contact_plocal',
 u'event_location',
 u'manufacturer_contact_address_2',
 u'manufacturer_g1_state',
 u'manufacturer_contact_phone_number',
 u'manufacturer_name',
 u'manufacturer_zip_code_ext',
 u'date_of_event',
 u'manufacturer_contact_extension',
 u'report_source_code',
 u'distributor_state',
 u'remedial_action',
 u'number_devices_in_event',
 u'manufacturer_contact_zip_ext',
 u'date_received',
 u'manufacturer_g1_zip_code_ext',
 u'mdr_text',
 u'reprocessed_and_reused_flag',
 u'type_of_report',
 u'manufacturer_contact_l_name',
 u'manufacturer_contact_coun

In [27]:
dicts =[]

In [28]:
##dicts = [r.json()['results'] for r in responses ]
#each dict here is one event
for r in responses:
    step=r.json()
    for i in range(100):
        dicts.append(step['results'][i])


In [9]:
#result = pd.concat(frames)
#doesn't concat usefully here, try again
adversedf = pd.DataFrame(columns=['num_people', 'generic_name', 'event_type', 'event_location', 'manufacturer'])

In [826]:
problemdf = pd.DataFrame(columns=['num_people', 'generic_name', 'event_type', 'event_location', 'manufacturer'])

In [10]:
def get_generic(dicts):
    index = 0
    for dict in dicts:
            if dict[0]['adverse_event_flag'] == "Y":
                adversedf.loc[index, 'num_people'] = 1
                #if dict[0]['number_patients_in_event'] == "" or isnan(dict[0]['number_patients_in_event']):
                #    adversedf.loc[index, 'num_people'] = 1
                #else: 
                #    adversedf.loc[index, 'num_people'] = (dict[0]['number_patients_in_event'])
                adversedf.loc[index, 'generic_name'] = str(dict[0]['device'][0]['generic_name'])
                adversedf.loc[index, 'event_type'] = str(dict[0]['event_type'])
                adversedf.loc[index, 'event_location'] = str(dict[0]['event_location'])
                adversedf.loc[index, 'manufacturer'] = str(dict[0]['device'][0]['manufacturer_d_name'])
                print "adverse"
            index+=1
           
            #print dicts[0][0]['number_patients_in_event']

In [926]:
def get_problems(dicts):
    index = 0
    for dict in dicts:
            if dict['product_problem_flag'] == "Y":
                problemdf.loc[index, 'num_people'] = 1
                #if dict[0]['number_patients_in_event'] == "" or isnan(dict[0]['number_patients_in_event']):
                #    adversedf.loc[index, 'num_people'] = 1
                #else: 
                #    adversedf.loc[index, 'num_people'] = (dict[0]['number_patients_in_event'])
                problemdf.loc[index, 'generic_name'] = str(dict['device'][0]['generic_name']).encode('utf-8', 'ignore').strip()
                problemdf.loc[index, 'event_type'] = str(dict['event_type'])
                problemdf.loc[index, 'event_location'] = str(dict['event_location'])
                problemdf.loc[index, 'manufacturer'] = str(dict['device'][0]['manufacturer_d_name'])
            index+=1

In [999]:
#problemdf.describe()
problemdf.head()
problemdf.to_csv("problemsample.csv")

In [927]:
get_problems(dicts)
#problemdf.loc[0,'manufacturer']

In [598]:
#newproblemdf = pd.read_csv("problemsample.csv")

In [963]:
problemdf.describe()

,num_people,generic_name,event_type,event_location,manufacturer
count,3551,3551,3551,3551,4587
unique,1,952,5,16,624
top,1,,Malfunction,,BAXTER HEALTHCARE
freq,3551,223,2919,2375,390


In [701]:
#sorted(newmfrnames)

In [ ]:
fuzz.token_set_ratio("VARIAN MEDICAL SYSTEMS", "AMERICAN MEDICAL SYSTEMS")

In [990]:
newfrmnames = sorted(list(problemdf['manufacturer']))
#newmfrnames

In [934]:
testset = sorted(list(problemdf['manufacturer']))

In [992]:
#testset[2]
len(set(newmfrnames))
#newmfrnames[0]

415

In [942]:
'MEDTRONIC PUERTO RICO OPERATIONS CO.' in list(testset)
len(testset)

3551

In [932]:
problemdf.head(10)

,num_people,generic_name,event_type,event_location,manufacturer
0,1,NEUROSTIMULATOR,Other,,MEDTRONIC INC.
1,1,"INSTRUMENT, ULTRASONIC SURGICAL",Malfunction,,"ETHICON ENDO-SURGERY, LLC."
3,1,INSULIN INFUSION PUMP / SENSOR AUGMENTED,Malfunction,I,MEDTRONIC PUERTO RICO OPERATIONS CO.
4,1,LEAD MODEL 302,Malfunction,OTHER,CYBERONICS INC
5,1,IMPLANTABLE PACEMAKER PULSE GENERATOR,Injury,HOSPITAL,ST JUDE MEDICAL CARDIAC RHYTHM MANAGEMENT DIVI...
6,1,RADIATION TREATMENT PLANNING SYSTEM - MUJ,Malfunction,,"IMPAC MEDICAL SYSTEMS, INC."
9,1,INSULIN INFUSION PUMP,Malfunction,I,ANIMAS CORPORATION
12,1,"STIMULATOR, SPINAL-CORD, TOTALLY IMPLANTED FOR...",Malfunction,I,MEDTRONIC NEUROMODULATION
13,1,GLUCOSE MONITORING SYS/KIT,Malfunction,,"LIFESCAN EUROPE, A DIVISION OF CILAG GMBH INTL"
16,1,BLOOD GLUCOSE MONITORING TEST STRIPS - LFR,Malfunction,OTHER,ROCHE DIAGNOSTICS


In [687]:
here = problemdf['manufacturer']=='MEDTRONIC PUERTO RICO OPERATIONS CO.'

In [799]:
len(set(errs))
#here

144

In [682]:
"'MEDTRONIC PUERTO RICO OPERATIONS CO.'" in testset

False

In [965]:
problemdf.info()
#reps

In [943]:
#items = ["apple",  "banan", "banana", "apple", "bannna", "orange", "oranges","appl", "orang", "orange", "bannna", "banana"]
#items = newmfrnames
items=testset
#items = [ 'MEDTRONIC INC.', 'ALCON INDUSTRIES, INC.',
 #'ALCON LABORATORIES, INC.', 'MEDTRONIC INK' ,'MEDTRONIC PUERTO RICO OPERATIONS CO.']

In [984]:
##takes a long time
reps = dict()
def clean_manufs(items):

    for hold in items:
        if hold in reps.keys():
            pass
        else:
            for compare in items:
                if compare in reps.keys():
                    pass
        ###make this higher than 90?
                elif fuzz.token_set_ratio(hold, compare) > 90:
                    print"set " + compare + " to " + hold
                    reps[compare] = hold 
                
    return reps

In [985]:
len(reps)

0

In [959]:
errs=[]
for index, item in enumerate(problemdf['manufacturer']):
    try:
        problemdf.loc[index, 'manufacturer'] = reps[item]
    except KeyError:
        errs.append(item)

In [727]:
#items = newmfrnames
#reps = {
#  item2: item1 
#  for item1 in items 
#  for item2 in items 
#  if fuzz.token_set_ratio(item1, item2) > 80
#}



In [960]:
len(set(errs))

2

In [982]:
#errs


In [649]:
'MEDTRONIC PUERTO RICO OPERATIONS CO.' in problemdf['manufacturer']
#reps.values()

False

In [823]:
len(reps)

393

In [624]:
#sorted(set(newmfrnames))
#521 at 90%
#415 at 80%

In [586]:
for item in newmfrnames:
    for i in range (0, len(newmfrnames)-1):
        compare = newmfrnames[i]
        if fuzz.token_set_ratio(item, compare) > 80:
            newmfrnames[i] = item

In [593]:
#takes a while to run 
for name in sorted(set(newmfrnames)):
    print name
    for index, item in enumerate(problemdf['manufacturer']):
        if fuzz.token_set_ratio(name, item) > 80:
            problemdf.loc[index, 'manufacturer'] = name

       

In [ ]:
fuzz.token_set_ratio()

In [833]:
#adversedf.head()
#testroup = adversedf.groupby('manufacturer')

In [834]:
#testroup.head()
#testroup.describe()

In [102]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [70]:
from bokeh.charts import Bar

In [19]:
adversedf.to_csv("sampledata.csv")

In [967]:
#adversedf.loc[1, 'num_people'] = 20
#adversedf
#adversedf.loc[1, 'num_people'] = 50

In [835]:
#get_generic(dicts)

In [900]:
#dicts[0][0].keys()

In [156]:
#result.info()

In [899]:
dicts[0]['device']

[{u'baseline_510_k__exempt_flag': u'',
  u'baseline_510_k__flag': u'',
  u'baseline_510_k__number': u'',
  u'brand_name': u'RESTORE ADVANCED',
  u'catalog_number': u'',
  u'date_received': u'20080814',
  u'date_removed_flag': u'V',
  u'device_age_text': u'',
  u'device_availability': u'Yes',
  u'device_evaluated_by_manufacturer': u'',
  u'device_event_key': u'1122358',
  u'device_operator': u'LAY USER/PATIENT',
  u'device_report_product_code': u'LGW',
  u'device_sequence_number': u'1',
  u'generic_name': u'NEUROSTIMULATOR',
  u'implant_flag': u'Y',
  u'lot_number': u'',
  u'manufacturer_d_address_1': u'RETURN FINISHED GOODS',
  u'manufacturer_d_address_2': u'ATTN RGR 2240 WOODALE DR',
  u'manufacturer_d_city': u'MOUNDS VIEW',
  u'manufacturer_d_country': u'US',
  u'manufacturer_d_name': u'MEDTRONIC INC.',
  u'manufacturer_d_postal_code': u'',
  u'manufacturer_d_state': u'MN',
  u'manufacturer_d_zip_code': u'55112',
  u'manufacturer_d_zip_code_ext': u'',
  u'model_number': u'',
  u'open

In [64]:
len(result['serious'])

5000

In [47]:
eventdf = pd.DataFrame(columns=['num_people', 'generic_name', 'event_type', 'event_location', 'manufacturer', 'source'])

In [901]:
index=0
for item in dicts:
    #want ability to select device type here but bokeh won't plot
       #if (item['device'][0]['generic_name']).find("CATHETER")==0:
    if item['number_patients_in_event'] == "" or isnan(item['number_patients_in_event']):
        eventdf.loc[index, 'num_people'] = 1
    else: 
        eventdf.loc[index, 'num_people'] = (item['number_patients_in_event'])
    if 'date_manufacturer_received' in item.keys():
        eventdf.loc[index, 'date_manufacturer_received'] = str(item['date_manufacturer_received'])
    else:
        eventdf.loc[index, 'date_manufacturer_received'] = "NA"
    if 'date_report' in item.keys():
        eventdf.loc[index, 'date_report'] = str(item['date_report'])
    else:
        eventdf.loc[index, 'date_report'] = "NA"
    if 'date_of_event' in item.keys():
        eventdf.loc[index, 'date_of_event'] = str(item['date_of_event'])
    else:
        eventdf.loc[index, 'date_of_event'] = "NA"
    if 'medical_specialty_description' in ((item['device'])[0]['openfda']).keys():
        eventdf.loc[index, 'specialty']=((item['device'])[0]['openfda'])['medical_specialty_description'].strip()
    else:
        eventdf.loc[index, 'specialty'] = "NA"
    if 'device_class' in ((item['device'])[0]['openfda']).keys():
        eventdf.loc[index, 'class']=((item['device'])[0]['openfda'])['device_class'].strip()
    else:
        eventdf.loc[index, 'class'] = "NA"
    if 'regulation_number' in ((item['device'])[0]['openfda']).keys():
        eventdf.loc[index, 'regulation_number']=((item['device'])[0]['openfda'])['regulation_number'].strip()
    else:
        eventdf.loc[index, 'regulation_number'] = "NA"
        
    eventdf.loc[index, 'generic_name'] = (item['device'][0]['generic_name']).encode('utf-8').replace(":", "")
    eventdf.loc[index, 'event_type'] = str(item['event_type'])
    eventdf.loc[index, 'event_location'] = str(item['event_location'])
    eventdf.loc[index, 'manufacturer'] = (item['device'][0]['manufacturer_d_name']).encode('utf-8').strip()
    eventdf.loc[index, 'source']=str(item['report_source_code'])
    eventdf.loc[index, 'adverse_event_flag'] = str(item['adverse_event_flag'])
    eventdf.loc[index, 'date_received'] = str(item['date_received'])
    

    index+=1

In [978]:
eventdf.describe()

,num_people,generic_name,event_type,event_location,manufacturer,source,specialty,adverse_event_flag,date_manufacturer_received,date_of_event,date_received,date_report,class,regulation_number
count,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
unique,1,1371,5,16,1157,4,20,3,2428,2391,2463,2513,7,307
top,1,,Malfunction,,MEDTRONIC PUERTO RICO OPERATIONS CO.,Manufacturer report,Unknown,N,NA,NA,20140113,NA,2,
freq,5000,348,3015,3312,176,4718,1311,3117,344,1128,14,301,2819,1311


In [ ]:
#sum(eventdf[eventdf['class']=="3"])
#add ['baseline_510_k__exempt_flag']??

In [987]:
testset2 = sorted(list(eventdf['manufacturer']))
len(set(testset2))

776

In [993]:
clean_manufs(testset2)
#make it clean_manufs(items, reps)

{'1219655 DEPUY-RAYNHAM, A DIV. OF DEPUY ORTHOPAEDIC': '1219655 DEPUY-RAYNHAM, A DIV. OF DEPUY ORTHOPAEDIC',
 '1818910 DEPUY ORTHOPAEDICS, INC.': '1818910 DEPUY ORTHOPAEDICS, INC.',
 '3003895575 DEPUY FRANCE S.A.S.-SAINT PRIEST': '3003895575 DEPUY FRANCE S.A.S.-SAINT PRIEST',
 '3006356043 JOHNSON & JOHNSON MEDICAL': '3006356043 JOHNSON & JOHNSON MEDICAL',
 '3600-C.R. BARD ACCESS SYSTEMS PRODUCTS': '3600-C.R. BARD ACCESS SYSTEMS PRODUCTS',
 '3M ESPE DENTAL PRODUCTS': '3M ESPE DENTAL PRODUCTS',
 '8010379 DEPUY INTL., LTD': '8010379 DEPUY INTL., LTD',
 '8010379 DEPUY INTL., LTD.': '8010379 DEPUY INTL., LTD',
 '9616671 DEPUY (IRELAND)': '9616671 DEPUY (IRELAND)',
 'A/O SYNTHES': 'A/O SYNTHES',
 'ABBOTT DIABETES CARE': 'ABBOTT DIABETES CARE',
 'ABBOTT DIAGNOSTICS DIVISION': 'ABBOTT DIAGNOSTICS DIVISION',
 'ABBOTT GERMANY': 'ABBOTT GERMANY',
 'ABBOTT LABORATORIES': 'ABBOTT LABORATORIES',
 'ABBOTT LABOROTORIES': 'ABBOTT LABORATORIES',
 'ABBOTT MANUFACTURING, INC.': 'ABBOTT MANUFACTURING, INC.

In [994]:
len(reps)

774

In [995]:
errs=[]
for index, item in enumerate(eventdf['manufacturer']):
    try:
        eventdf.loc[index, 'manufacturer'] = reps[item]
    except KeyError:
        errs.append(item)

In [874]:
((dicts[1]['device'])[0]['openfda'])['medical_specialty_description']

u'Unknown'

In [997]:
set(errs)

{'', '*'}

In [998]:
eventdf.to_csv("largesample.csv")

In [895]:
pony = set(eventdf['generic_name'])

In [966]:
#sorted(pony)

In [39]:
count = 0
for i in range(len(dicts)):
    if dicts[i]['report_source_code'] == 'Voluntary report':
        count += 1
    
    

In [76]:

p = figure(plot_width=400, plot_height=400)

##output_file("line_bar.html")

# add a line renderer
p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)

# setting bar values
h = np.array([2, 8, 5, 10, 7])

# Correcting the bottom position of the bars to be on the 0 line.
adj_h = h/2

# add bar renderer
p.rect(x=[1, 2, 3, 4, 5], y=adj_h, width=0.4, height=h, color="#CAB2D6")

# Setting the y  axis range   
#p.y_range = Range1d(0, 12)

p.title = "Line and Bar"

show(p)

NameError: name 'show' is not defined

In [40]:
count
#this is 2.3% of events

116

In [893]:
#import bokeh
#bokeh.__version__
#from bokeh.charts import Line
#from bokeh.plotting import output_file,show
#adversedf.groupby(['manufacturer']).sum()
#p = Bar(adversedf, label='num_people', values='manufacturer', agg='sum',
#        title="Average MPG by YR")

#output_file("bar.html")

#show(p)

In [77]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from bokeh.charts import Line
from bokeh.plotting import show, output_file
from bokeh.charts import Chart, Line
xyvalues = OrderedDict(
    python=[2, 3, 7, 5, 26, 221, 44, 233, 254, 265, 266, 267, 120, 111],
    pypy=[12, 33, 47, 15, 126, 121, 144, 233, 254, 225, 226, 267, 110, 130],
    jython=[22, 43, 10, 25, 26, 101, 114, 203, 194, 215, 201, 227, 139, 160],
)

# any of the following commented are valid Line inputs
xyvalues = pd.DataFrame(xyvalues)
#xyvalues = xyvalues.values()
#xyvalues = np.array(xyvalues.values())

output_file("lines.html", title="line.py example")
chart = Line(xyvalues, title="Lines", ylabel='measures', legend=True)
show(chart)

ERROR:/Users/taftmphee/anaconda/lib/python2.7/site-packages/bokeh/core/validation/check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 18b3140c-6e0b-4509-8a4e-b87f1f01019f


In [78]:
from bokeh.plotting import figure, output_file, show
output_file("test.html")
p = figure()
p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)
show(p)


ERROR:/Users/taftmphee/anaconda/lib/python2.7/site-packages/bokeh/core/validation/check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 18b3140c-6e0b-4509-8a4e-b87f1f01019f
